In [11]:
# All the imports you will need in the whole lab
from skimage.feature import greycomatrix, greycoprops
from skimage import io
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import os
import numpy as np

%matplotlib qt

In [12]:
def getVerticalProjectionProfile(image):
    image[image == 0] = 1
    image[image == 255] = 0
  
    vertical_projection = np.sum(image, axis = 0)
    return vertical_projection.mean()

In [13]:
def getHorizontalProjectionProfile(image):
    image[image == 0] = 1
    image[image == 255] = 0
  
    horizontal_projection = np.sum(image, axis = 1) 
    return horizontal_projection.mean()

In [14]:
def get_glcm_features(gray_scale_img):
    glcm = greycomatrix(gray_scale_img, distances=[1], angles=[0], levels=256)
    contrast = greycoprops(glcm, prop='contrast')
    homogeneity = greycoprops(glcm, prop='homogeneity')
    
    return contrast, homogeneity

In [15]:
def get_fname_images_tuple(directory):
    fnames = os.listdir(directory)
    to_return = []
    for fn in fnames:
        if fn[-3:] == 'jpg':
            path = os.path.join(directory, fn)
            gray_scale_image = (rgb2gray(io.imread(path)) * 255).astype(np.uint8)
            ###################################################################################
            ###################################################################################
            img_2 = gray_scale_image.copy()
            ###################################################################################
            ###################################################################################
            if gray_scale_image.mean() < 150:
                # THIS MEANS IMAGE IS INVERTED
                img_2[gray_scale_image * 255 > 170] = 0
                img_2[gray_scale_image * 255 <= 170] = 255
            else:
                img_2[gray_scale_image * 255 > 170] = 255 
                img_2[gray_scale_image * 255 <= 170] = 0 
            ###################################################################################
            ###################################################################################
            to_return.append((fn, img_2))
        else:
            print('This file has been skipped', fn)
    return to_return

In [16]:
def plot_2d_features_with_names(names_fts_list, xlabel, ylabel, zlabel, colors, class_names):   
    fig = plt.figure()
    # ax = fig.add_subplot()
    ax = fig.add_subplot(111, projection='3d')
    
    for class_i in range(len(names_fts_list)):
        x = [names_fts[1][0] for names_fts in names_fts_list[class_i]]
        y = [names_fts[1][1] for names_fts in names_fts_list[class_i]]
        z = names_fts_list[class_i][0][2]
        
        # ax.scatter(x, y, color=colors[class_i], label=class_names[class_i])
        ax.scatter(x, y, z, color=colors[class_i], label=class_names[class_i])
        
    ax.set_title('Training Data')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_zlabel(zlabel)

    handles, labels = ax.get_legend_handles_labels()
    unique = list(set(labels))
    handles = [handles[labels.index(u)] for u in unique]
    labels = [labels[labels.index(u)] for u in unique]

    ax.legend(handles, labels)

    plt.show()

In [17]:
def extract_features_from_image(number_of_fonts):
    extracted_features = []
    for folder_num in range(1, number_of_fonts + 1):
        folder_path = ".\\ACdata_base\\" + str(folder_num)
        fn_img_list = get_fname_images_tuple(folder_path)

        fn_ft_list = []
        for fn, img in fn_img_list:
            fn_ft_list.append((fn, get_glcm_features(img), getVerticalProjectionProfile(img)))
        
        extracted_features.append(fn_ft_list)
    return extracted_features

In [18]:
# STILL NOT USED
def calculateDistance(x1, x2):
    distance = np.linalg.norm(np.asarray(x1) - np.asarray(x2))
    return distance

In [19]:
# STILL NOT USED
def MinimumDistanceClassifier(test_point, training_features):
    distances = []
    for class_i in range(len(training_features)):
        a_class = training_features[class_i]
        class_mean = []
        class_mean = np.mean(a_class, axis=0)
        distances.append(calculateDistance(test_point, class_mean))

    classification = np.argmin(distances)
    
    return classification

In [20]:
number_of_fonts = 9
colors = ['darkblue', 'green', 'blue', 'yellow', 'aqua', '#b03060', 'hotpink', 'navajowhite', 'red']
font_names = ['diwani', 'naskh', 'parsi', 'rekaa', 'thuluth', 'maghribi', 'kufi', 'mohakek', 'Squar-kufic']

font_features = extract_features_from_image(number_of_fonts)
    
plot_2d_features_with_names(font_features, 'contrast', 'homogeneity', 'Vertical Projection Profile Mean', colors, font_names)

<ipython-input-15-0fcbede57a2a>:7: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  gray_scale_image = (rgb2gray(io.imread(path)) * 255).astype(np.uint8)
